In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_binary
import csv
import re

In [3]:
driver = webdriver.Chrome()
rent_category = []
num = 1
length = len(rent_category)

while length < 1500:
    print("numの現在の数：ページ数",num)
    print("現在の取得カテゴリ番号",rent_category)
    print("rent_category:length",length)
    time.sleep(5)
    driver.get('https://www.f-takken.com/freins/rent/houses/area/items?listtype=houses&lang=ja&location=area&locate%5B0%5D=40131&locate%5B1%5D=40132&locate%5B2%5D=40133&locate%5B3%5D=40134&locate%5B4%5D=40135&locate%5B5%5D=40136&locate%5B6%5D=40137&order1=nn&order2=&limit=20&data_21=0&data_22=0&data_30=0&data_31=0&data_486=&data_486_code=&data_487=&data_487_code=&page={}&currentTabIndex=0'.format(num))
    time.sleep(10)
    # 検索先のページのHTMLを取得
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2)
    
    # 表示されているカテゴリの番号を全て記憶 
    for element in soup.find_all("li",class_ = "item list-tpl"):
        data_id = element.get("data-id")
        rent_category.append(data_id)
        length = len(rent_category)
        rent_category = list(set(rent_category))    
    num = num+1
print(rent_category)



KeyboardInterrupt: 

In [ ]:
# 1000個のカテゴリ番号を保存できるまで、次のページに遷移する TODO ○
# driverのpage = {}を変更していく TODO ○
# whileでrent_categoryが1000より少ない時までnumを増やしてpageにformatする ○
#取得の繰り返し ○

#全て取得したら、driverに詳細のページのURLでカテゴリ番号を追記して表示 ○
#表示したら、必要な情報を取得、保存

In [ ]:

count = 0
# 書き込み
with open('RENT.csv', 'w') as csv_file:
    fieldnames = ['家賃','物件番号','情報更新日','次回更新予定日','所在地','建物名','沿線・駅・交通','専有面積','専用庭面積','間取り','間取り内訳','バルコニー','サービスルーム数'
                 ,'小学校区','中学校区','駐車場','駐車場：形式','駐車場：状況','駐車場備考','駐車場敷金','敷金','礼金','精算方式','保証金','保証金償却','管理費','共益費'
                 ,'雑費','家賃保証会社','家賃保証・保証費用等','保険','更新料','その他一時金','施設費用','その他費用','賃貸借区分','契約期間','特優賃区分','管理員の勤務形態','敷地延長','建物構造'
                 ,'建物工法','最適用途','階建','階','築年月','現況','入居可能時期','設備','特記事項','備考','取引態様','担当者','担当者連絡先'] # 54
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

while len(rent_category) > count:
    tmp = []
    data_set = int(rent_category[count])
    time.sleep(5)
    driver.get('https://www.f-takken.com/freins/items/{}'.format(data_set))
    time.sleep(10)
    # 検索先のページのHTMLを取得
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2)
    count +=1

    # 賃料のみ最初に取得
    housePrice = soup.find("span",class_ = "price-area")
    if housePrice == None:
        continue
    price = re.sub(r"[^\d.]", "", housePrice.text)
    tmp.append(price)

    # 各種カテゴリーの値取得
    for category in soup.find_all("table",class_ = "detail-list"):
        time.sleep(1)
        for decom in category.find_all("td",class_ = "category" ):
            decom.decompose()
        for element in category.find_all("td"):
            tmp.append(element.text)
    print("tmpのlen：",len(tmp))
    
    # カテゴリの数が等しくない時、飛ばす
    if len(tmp)!= len(fieldnames):
        delete_ = 0
        delete_ += 1
        print("カテゴリの数が違うので飛ばします")
        print("飛ばした数",delete_)
        continue


    # 書き込み
    with open('RENT.csv', 'a') as csv_file:
        contents = {}
        writer = csv.writer(csv_file)
        writer.writerow(tmp)

print("成功しました")
